In [5]:
## clean
!find multiplier/ ! -name 'multiplier.circom' -type f -exec rm -f {} +
!find quadratic/ ! -name 'quadratic.circom' -type f -exec rm -f {} +
!rm -rf witness.json

# Introduction to ZK (Practice)

##### С помошью ZK алгоритмов, мы можем доказывать утверждения без раглашения части информации.
##### Доказательства строятся путем проведения какого-то вычисления.
##### Затем, данное доказательство может быть провалидировано с помощью верифаера.
##### **Q**: Какие утверждения можно доказывать?
##### **A**: В целом можно доказывать все что угодно, и, наверное, можно делать специализированные алгоритмы для конкретных доменов. Но самым распространенным вариантом выражения вычислений являются криптографические схемы (*Arithmetic circuits*). 

![Arithmetic circuit](images/1.png)

##### Выше пример простой криптографической схемы. Схемы состоят из переменных и констант. Они соединяются между собой операциями сложения и умножения (в рамках поля $F$). Переменные в таких вычислениях могут быть публичными и приватными - публичные известны всем, приватные известны только пруверу и применяются при создании доказательства.
##### На картинке выше схема состоит из двух переменных $x_1$, $x_2$ и константы $1$. Они соединяются через сложения и умножения, и в конечно итоге получается выражение $(x_1 + x_2) \cdot (x_2 + 1) \cdot x_2$.
##### Это простое выражение не несет какого-то специального смысла, однако для примера можно рассмотреть его так: пусть есть верифаер, который проверят, что юзер знает такие $x_1$ и $x_2$, что выражение выше обращается в какое-то число $C$ (и если проверка прошла успешно, то делает что-то хорошее для прувера).
##### Пусть $С = 10$. Тогда одно из решений это $x_1 = 4, x_2 = 1$. Также пусть $x_1, x_2$ - приватные переменные. Для построения доказательства, необходимо явно провести вычисления в этом графе, используя $x_1$ и $x_2$ и сверить после вычисления результат с константой $C$. $C$ в данном случае - публичный вход.
##### Верифаер получит пруф для проверки (какой-то набор чисел, сгенерированный прувером через умные zk алгоритмы). И подаст на вход функции верификации 2 вещи: пруф и все необходимые публичные перменные (в данном случае $C$).

##### Выше представлена тривиальная схема. Но через такие же гейты можно выразить более сложные вещи (например hash-функции).

##### Представьте, что верифаер теперь проверяет не знание чисел $x_1$ и $x_2$, а знание такого сообщения $m$, что $h = sha256(m)$, где $h$ - публичная переменная. Схематично все выглядит точно также, как в примере выше - но применений у этого уже гораздо больше.

# Circom

##### Конечно, в реальности никто не строит такие схемы руками. Есть различные ЯП/библиотеки/фреймворки с разными степенями абстракции, для генерации доказательств и верифаеров. Сегодня познакомимся с circom - довольно низкоуровневым фреймворком для построения криптографических схем.
#### [Документация](https://docs.circom.io/getting-started/installation/)
##### Для начала, поставим нужные зависимости.

In [ ]:
!curl --proto '=https' --tlsv1.2 https://sh.rustup.rs -sSf | sh # возможно придется запустить в терминале, чтобы повыбирать пункты во время установки

info: downloading installer

Welcome to Rust!

This will download and install the official compiler for the Rust
programming language, and its package manager, Cargo.

Rustup metadata and toolchains will be installed into the Rustup
home directory, located at:

  /Users/olegggatttor/.rustup

This can be modified with the RUSTUP_HOME environment variable.

The Cargo home directory is located at:

  /Users/olegggatttor/.cargo

This can be modified with the CARGO_HOME environment variable.

The cargo, rustc, rustup and other commands will be added to
Cargo's bin directory, located at:

  /Users/olegggatttor/.cargo/bin

This path will then be added to your PATH environment variable by
modifying the profile files located at:

  /Users/olegggatttor/.profile
  /Users/olegggatttor/.zshenv

You can uninstall at any time with rustup self uninstall and
these changes will be reverted.

Current installation options:


   default host triple: aarch64-apple-darwin
     default toolchain: stable (defa

In [1]:
# !git clone https://github.com/iden3/circom.git
# !cargo build --release
# !cargo install --path circom

##### Также надо поставить snarkjs, для валидации circom скриптов

In [ ]:
!npm install -g snarkjs

##### Проверям, что все работает

In [6]:
!circom --help

circom compiler 2.1.8
IDEN3
Compiler for the circom programming language

USAGE:
    circom [FLAGS] [OPTIONS] [--] [input]

FLAGS:
        --r1cs                                 Outputs the constraints in r1cs format
        --sym                                  Outputs witness in sym format
        --wasm                                 Compiles the circuit to wasm
        --json                                 Outputs the constraints in json format
        --wat                                  Compiles the circuit to wat
    -c, --c                                    Compiles the circuit to c
        --O0                                   No simplification is applied
        --O1                                   Only applies signal to signal and signal to constant simplification
        --O2                                   Full constraint simplification
        --verbose                              Shows logs during compilation
        --inspect                              Doe

##### Я буду дублировать код из файлов в ячейки для наглядности, но запускать будем файлы из multiplier/

##### Ниже представлен код из файла multiplier/multiplier.cicrom

##### Пройдемся по коду сверху вниз.
##### 1) На первой строке мы видим прагму версии компилятор, тоже самое мы обычно видим в солидити контрактах.
##### 2) Далее идет комментарий, он выделяется символами /* ... */ или // и игнорируется при компиляции.
##### 3) Затем мы встречаем слово **template**. Темплейты позволяют выделять общие части кода в отдельную сущность. Представьте, что мы строим большую схему и нам надо сделать много раз одно и то же умножение. Самым удобным вариантом будет вынести это в темплейт и переиспользовать по необходимости. (Как выглядит само переиспользование покажу позже.
##### 4) В теле темплейта первое, что мы видии это ключевое слово **signal**. Сигналы являются аналогами нод графа вычисление выше. После слова **signal** могут идти слова **input** или **output**. Сигналы с таким кодовым словом видны снаружи. Связывая **input** сигналы с другими сигналами, мы сможем получить **output** сигнал. Сигналов внутри темплейта без **input/output** ключевого слова не видны снаружи и могут использоваться только внутри темплейта. Последней компонентной является название сигнала, тут это *a, b, c*.
##### 5) На следующей строке мы видим констрейт (**Constraints**). Это связи для наших сигналов. Подробнее можно найти [тут](https://docs.circom.io/circom-language/constraint-generation/). Но если коротко, то такая конструкция позволяет связывать сигналы и накладывать ограничения на такие связи. Констрейты должны быть представлены в виде A*B + C = 0. Несколько синтаксичесмких конструкций связанных с этим:
##### 5.1) **a * b === С** - здесь **С** - какая-то константа или другой сигнал. Такой констрейт не соединяет сигналы, а проверяет, что равенство верно. Если при генерации доказательства такой констрейт не удволетворяется, то доказательство построить не получится.
##### 5.2) **C === a * b** - тоже самое, что в 5.1, части равенства можно менять местами
##### 5.3) **x <-- a * b** - стрелочка служит для создания связий между сигналами, при этом, данный синтаксис не генерирует и не является констрейтом (!!!), а лишь присваивает сигналу x новое значение a * b. 
##### 5.4) **a * b --> x** - тоже самое, что 5.3
##### 5.5) **x <== a * b** - это короткая запись двух выражений: **x <-- a * b** и **x === a * b**. Ровно такая запись встречается выше в примере. Чаще всего используется именно **<==**, а не **<--**, позже покажу чем **<--** может быть опасен и чем полезен. Можно писать стрелку и в другую сторону (**==>**).

##### По итогу, у нас есть темплейт Multiplier2, который принимает на вход два входа *a* и *b*, уможает их и результат умножения помещает в сигнал *c*.
##### Для генерации доказательств, нам нужно добавить точку входа для нагей схемы и скомпилировать ее. Добавим одну строку к скрипту выше:

##### Ключевое слово component позволяет инстанцировать темплейт - тут по сути мы просто говорим, что у нас будет component main и она будет равна Multiplier2. У main есть 2 входа *a* и *b* и на выходе мы получим выход *c*.
##### Пора компилировать! Но во что же преобразуется наш multiplier.circom? Компилятор circom генерирует *r1cs constraint system*. Используя код выше и опираясь на констрейты (===, <==, ==>) он генерирует специальный файл, который позволяет создать верифаер и генерировать доказательства. 
##### Итак, скомпилируем нашу схему такой командой:

In [8]:
!circom multiplier/multiplier.circom --r1cs --wasm --sym --output multiplier

template instances: 1
non-linear constraints: 1
linear constraints: 0
public inputs: 0
private inputs: 2
public outputs: 1
wires: 4
labels: 4
Written successfully: multiplier/multiplier.r1cs
Written successfully: multiplier/multiplier.sym
Written successfully: multiplier/multiplier_js/multiplier.wasm
Everything went okay


# Ура!
##### На выходе мы получили несколько файлов тк использовали несколько ключей при компиляции:
##### 1) multiplier.r1cs - основная часть, констрейты в бинарном формате. Не важно, что там внутри, но эта вещь необходимо для генерации верифаера и доказательств.
##### 2) multiplier.sym - файл для дебагинга TODO, но из важного там видно наши сигналы - main.a, main.b и main.c
##### 3) multiplier_js/multiplier.wasm - файл необходимый для генерации witness - об этом ниже.

#### Следующий шаг - генерация witness (свидетельств).
##### По сути это множество сигналов, используемых при генерации доказательства. Чтобы что-то доказывать, нам нужно присвоить всем сигналам конкретные значения. Однако, на самом деле, большинство сигналов вычисляются из других сигналов, и нам нужно задать значения только для входов - в данном случае *main.a* и *main.b*.

##### Создадим файл input.json и в json формате зададим значения для наших input сигналов.

In [11]:
!cd multiplier/multiplier_js && touch input.json && echo '{"a": "3", "b": "5"}' > input.json

##### Для генерации witness воспользуемся файлом ./multiplier_js/multiplier.wasm

In [12]:
!cd multiplier/multiplier_js && node generate_witness.js multiplier.wasm input.json witness.wtns

##### На выходе мы получили файлы witness.wtns и witness_calculator.js в папке multiplier_js. Посмотрим, что лежит в witness.wtns. Для этого надо превратить этот файл в читаемый json

In [13]:
!snarkjs wtns export json multiplier/multiplier_js/witness.wtns && cat witness.json

[
 "1",
 "15",
 "3",
 "5"
]

##### И здеь мы можем увидеть значения всех посчитанных сигналов! Первый элемент всегда 1. Затем 15 - это результат, c. 3 - a, 5 - b. Итого: [1, c, a, b]

##### Теперь у нас есть все для создания нашего доказательства: witness.wtns и multiplier.r1cs
##### Но что вообще мы доказываем? В нашем примере оба входа a и b являются приватными, это дефолтные значения при создании компоненты main. Известен будет только выход нашего вычисления. Таким образом, мы будем доказывать, что знаем два числа *a* и *b* такие, что их произведение равно *c*.

##### Есть много различных алгоритмов используемых для генерации доказательств - возьмем для примера Groth16.
##### Первый шаг - trusted setup. Это процедура генерации определенных параметров для создания доказательств. Важно: нечестная генерация таких параметров может позволить доказывать неверные утверждения, так что это компонента генерации доказательства очень важна.
##### Trusted setup состоит из двух частей
##### 1) Процедура powers of tau - это часть одинаковая для всех алгоритмов
##### 2) Phase 2 - эта часть зависит от алгоритма (у нас Groth16)
### Начнем с Powers Of Tau. Инициируем церемонию:

In [15]:
import os
os.chdir("multiplier")

In [16]:
!snarkjs powersoftau new bn128 12 pot12_0000.ptau -v

[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Calculate Initial Hash: tauG2
[DEBUG] snarkJS: Calculate Initial Hash: alphaTauG1
[DEBUG] snarkJS: Calculate Initial Hash: betaTauG1
[DEBUG] snarkJS: Blank Contribution Hash:
		786a02f7 42015903 c6c6fd85 2552d272
		912f4740 e1584761 8a86e217 f71f5419
		d25e1031 afee5853 13896444 934eb04b
		903a685b 1448b755 d56f701a fe9be2ce
[INFO]  snarkJS: First Contribution Hash:
		9e63a5f6 2b96538d aaed2372 481920d1
		a40b9195 9ea38ef9 f5f6a303 3b886516
		0710d067 c09d0961 5f928ea5 17bcdf49
		ad75abd2 c8340b40 0e3b18e9 68b4ffef


##### Теперь, законтрибутим в церемонию - в реальном кейсе, в процедуру контрибутят множество разных участников - это делает параметры безопасными. Никто не сможет генерировать ложные пруфы. Но у нас, сделаем сами 1 контрибьют:

P.S Возможно нужно будет ввести текст в качестве соли, будет удобнее это сделать в отдельном терминале.

In [17]:
!snarkjs powersoftau contribute pot12_0000.ptau pot12_0001.ptau --name="First contribution" -v

Enter a random text. (Entropy): 

##### Мы получим файл pot12_0001.ptau из файла pot12_0000.ptau
##### pot12_0001.ptau - наши параметры с первой стадии
### Phase 2

##### Начинаем генерацию этой фазы:

In [18]:
!snarkjs powersoftau prepare phase2 pot12_0001.ptau pot12_final.ptau -v

[DEBUG] snarkJS: Starting section: tauG1
[DEBUG] snarkJS: tauG1: fft 0 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 0 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 1 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 2 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix start: 0/1
[DEBUG] snarkJS: tauG1: fft 3 mix end: 0/1
[DEBUG] snarkJS: tauG1: fft 4 mix start: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix start: 1/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 0/2
[DEBUG] snarkJS: tauG1: fft 4 mix end: 1/2
[DEBUG] snarkJS: tauG1: fft  4  join: 4/4
[DEBUG] snarkJS: tauG1: fft 4 join  4/4  1/1 0/1
[DEBUG] snarkJS: tauG1: fft 5 mix start: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 2/4
[DEBUG] snarkJS: tauG1: fft 5 mix start: 3/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 0/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 1/4
[DEBUG] snarkJS: tauG1: fft 5 mix end: 3/4
[DEBUG] snarkJS: tauG1: fft 5 m

##### И еще пара команд:

In [19]:
!snarkjs groth16 setup multiplier.r1cs pot12_final.ptau multiplier_0000.zkey

[INFO]  snarkJS: Reading r1cs
[INFO]  snarkJS: Reading tauG1
[INFO]  snarkJS: Reading tauG2
[INFO]  snarkJS: Reading alphatauG1
[INFO]  snarkJS: Reading betatauG1
[INFO]  snarkJS: Circuit hash: 
		c89b966f 14cca796 a860e988 5949467b
		3985a836 0a054dea 00426027 f5d784f7
		07efc08b fb1a95ab b3bf50dd c40f523d
		1bc40636 3332c2f7 7fbd13b7 7153aaf5


In [41]:
!snarkjs zkey contribute multiplier_0000.zkey multiplier_0001.zkey --name="1st Contributor Name" -v

Enter a random text. (Entropy): 

In [20]:
!snarkjs zkey export verificationkey multiplier_0001.zkey verification_key.json

[INFO]  snarkJS: EXPORT VERIFICATION KEY STARTED
[INFO]  snarkJS: > Detected protocol: groth16
[INFO]  snarkJS: EXPORT VERIFICATION KEY FINISHED


### Церемония завершена! Теперь мы можем сгенерировать доказательство.
##### Для этого будем использовать:
##### 1)witness.wtns - значения для всех сигналов
##### 2)multiplier_0001.zkey - параметры со второй фазы

In [21]:
!snarkjs groth16 prove multiplier_0001.zkey multiplier_js/witness.wtns proof.json public.json

##### На выходе мы получили 2 файла: proof.json и public.json
##### proof.json - доказательство, вернемся к нему позже.
##### public.json - все публичные сигналы нашей схемы - в данном случае это только out, результат произведения.

In [22]:
!cat public.json

[
 "15"
]

##### Теперь мы можем проверить наше доказательство (пока НЕ в виде смарт контракта):

In [23]:
!snarkjs groth16 verify verification_key.json public.json proof.json

[INFO]  snarkJS: OK!


##### Как мы видим, никаких секретов не раскрыто!
##### Давайте сделаем магию и сгенерируем контракт верифаер:

In [24]:
!snarkjs zkey export solidityverifier multiplier_0001.zkey MultiplyVerifier.sol

[INFO]  snarkJS: EXPORT VERIFICATION KEY STARTED
[INFO]  snarkJS: > Detected protocol: groth16
[INFO]  snarkJS: EXPORT VERIFICATION KEY FINISHED


##### Мы получили файл MultiplyVerifier.sol!
##### В нем есть одна view функция - verifyProof. В качестве аргументов она принимает ранее снегенрированное доказательство и публичный инпуты (в нашем случае это результат умножения - 15), и возвращает True только если доказательство верное. Можем использовать это в наших контрактах как нам нужно!
##### Для генерации параметров для солидити можно использовать эту команду:

In [25]:
!snarkjs generatecall

["0x163521351ce2b84cf612751207fed793edbe3f4addde5c0bba5f9422b8b66841", "0x1f420a55955f279321243d9123a45479448535ac47442cb70b6040f557983c19"],[["0x03be98cf43eb2c4eb57cbfba74147a4a1f829bdca9e8fd38ec634d6e665e7983", "0x1bd249ae67844b4a54c1139199d302d35a50ed5990e9415187c97e07fb22ba1c"],["0x27ada01bc98b4ce9736b97d6b4b2a10f0a0e24083e75f6d62038c5019730eee0", "0x240cc052d72dd32c33be203f99acc55f33d0734db46d0d146d3e8063bad18858"]],["0x252103536b2d5e19caef0f4189956a7e16744af10f2d494e18fe0b8d788a9988", "0x0aefc6f756665fd21712315b4495762e1b19780c739101be45be9b7e64d8991d"],["0x000000000000000000000000000000000000000000000000000000000000000f"]


### <-- vs <==
##### Почему важно в большинстве мест использовать <==, а не <--? 
##### <-- не является по факту констрейтом, а лишь соединят сигналы между собой. Это позволяет писать более гибкие вещи, ниже пример проверки *in* на равенство нулю. Возвращаем 1, если *in == 0*, 0 иначе.
##### Работая с сигналом *inv* мы используем оператор **<--**. Как мы видим, выражение справа не в форме A * B + C, a использует тернарный оператор и даже деление.

##### Но для генерации доказательства сам код используется только для вычисления witness значений сигналов.
##### Допустим мы генерируем доказательство для входа 10.
##### На выходе у нас будет такой witness файл: **[1, out, in, inv] == [1, 0, 10, 1/10 % p]**
##### Ничего не мешает нам подменить значения в этом сведетельстве. Если какой-то сигнал не будет защищен достаточными констрейтами, это позволит сгенерировать верное доказательство для ложного входа. Если же писать код правильно, то доказательство сгенерировать не получится.
##### Именно поэтому в темплейте выше есть последняя строка **in*out === 0**. Оно проверит, что либо наш инпут 0, либо *out* это 0 - то есть наш *in* не 0 и мы верно посчитали его обратный элемент.
##### Вот пример уязвимой реализации:

##### Код выше не обладает никаким констрейтами, по сути любые значения сигналов в witness подойдут для генерации доказательств. Хотя сам код - по сути делает то, что нужно, но не влияет на доказательство.

### Более сложная схема
##### Ниже пример схемы, для генерации доказательста того, что пруверу известны корни некоторого квадратного уравнения.
#####

In [32]:
os.chdir("../quadratic")
!circom quadratic.circom --r1cs --wasm --sym

template instances: 2
non-linear constraints: 3
linear constraints: 0
public inputs: 3
private inputs: 1
public outputs: 0
wires: 7
labels: 14
Written successfully: ./quadratic.r1cs
Written successfully: ./quadratic.sym
Written successfully: ./quadratic_js/quadratic.wasm
Everything went okay


##### Пример выше принимает на вход массив сигналов *coeffs* и сигнал *x*.
##### *Coeffs* - публичные входы, их нужно будет подать верифаеру на вход при верификации. *x* же является скрытым входом и не будет известен верифаеру.
##### Главной точкой входа является *main*. Там мы явно указываем, что *coeffs* - публинчые входы.
##### *SolveQuadraticExpression* внутри себя инициализирует несколько компонент для умножения: *x2*, *ax2* и *bx*. Входные сигналы соедняются с входами компонент через оператор *<==*, то есть запись *x2.a <== x* передают сигнал *x* на вход *x2* в качестве входа *a*. При использовании компоненты, необходимо инициализировать ее все **input** входы.
##### Выходы компонент можно переиспользовать: для вычисления *a * x ^ 2* мы передаем выход компоненты *x2* на вход (тк хотим перемножить коэффициент *a* и *x^2*).
##### Для вычисления результата выражения мы используем переменную *var result*. **var** не является сигналом, и служит для простой записи более сложных выражений. Поэтому после вычисления значения выражения, мы явно задаем констрейт: *result === 0*, означающий, что *x* является корнем выражения.
##### На самом деле, компоненты *Multiplier2()* можно было опустить и вычислить все через var. Circom позволяет выносить сложные вычисления вне констрейтов и верификации, но необходимо правильно сравнить результаты этих вычислений.
##### Так бы могла выглядеть упрощенная версия: